# Trigram으로 나눠진 라벨링된 데이터를 불러와 벡터화 시키기

In [10]:
import pandas as pd
import numpy as np
from JamoSplit import jamo_combine, jamo_split

In [11]:
data = pd.read_csv("./preprocessing_labeled_data.csv", header=None)

# ==== input에 맞게 데이터 수정 ====

In [12]:
# int형으로 문제가 발생하여 전부 str 타입으로 설정
data[0] = data[0].astype("str")
data[1] = data[1].astype("str")
data[2] = data[2].astype("str")

In [13]:
# 빈칸이 nan되는 문제 다시 공백으로 체인지
data[0] = data[0].apply(lambda x: " " if x == 'nan' else x)
data[1] = data[1].apply(lambda x: " " if x == 'nan' else x)
data[2] = data[2].apply(lambda x: " " if x == 'nan' else x)

In [14]:
# 데이터 한 column으로 합치기
data['trigram'] = data[0] + "$" + data[1] + "$"+ data[2]
del data[0], data[1], data[2] # 합친후에 삭제

# 자모분리
data['trigram'] = data['trigram'].apply(lambda x: jamo_split(x))
# ㅂㅏ_ㅂㅗ_ 가 한 word가 될 수 있도록 만들어주는 과정
data['trigram'] = data['trigram'].apply(lambda x: x.split("$"))

In [15]:
data.head() # 3column이 label이다.

,3,trigram
0,1,"[ㅁㅗㅅㅎㅏ_ㄴㅡㄴ, ㄱㅘㄴㅈㅗㅇ, ㅠㅠ]"
1,1,"[ , ㅅㅟ_ㅂㅓㄹ, ]"
2,0,"[ , ㅇㅠ_ㅎㅡㅇ, ㄱㅏ_ㅈㅣ_ㄱㅗ_]"
3,0,"[ㅈㅓㅈㅂㅕㅇ, ㅈㅓㅈㄲㅗㄱㅈㅣ_, ]"
4,1,"[ㅁㅣㄴㅈㅜ_ㅋㅟ_, ㄷㅗㄹㅇㅏㅆㄴㅏ_, ㅁㅣㄴㅈㅜ_ㅍㅐㄴㄷㅡㄹㅇㅡㄴ]"


In [16]:
# fasttext 모델 불러오기
from gensim.models import FastText

embedding_model = FastText.load("./gensim_festtext.model")

In [17]:
# 각 단어를 벡터화 시켜주는 과정 3 x 50(embedding dimension) 
data['trigram'] = data['trigram'].apply(lambda x: [embedding_model[_] for _ in x])

/Users/daier/anaconda3/envs/testEnv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [19]:
data.head()

,3,trigram
0,1,"[[-1.953795, -1.1928164, -0.18564849, -0.40059..."
1,1,"[[0.01287517, 0.013801623, 0.017847467, -0.012..."
2,0,"[[0.01287517, 0.013801623, 0.017847467, -0.012..."
3,0,"[[-0.41459364, 0.14193316, -0.7080049, 0.40626..."
4,1,"[[-0.17480232, -0.062112387, -0.6108945, 0.764..."


In [18]:
# 벡터화된 데이터로 저장
data.to_json("./labeled_data.json")